<a href="https://colab.research.google.com/github/grettadarmstrong/github-slideshow/blob/master/AIBootCampPandasIntro3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Wrangling with pandas - Combining Datasets: Aggregation and Grouping

**Authors**: 
- Dr. Jany Chan, The Ohio State University
- Dr. Chaitanya Kulkarni, The Ohio State University
- Prof. Raghu Machiraju, The Ohio State University

---
## Context
The material here was developed by the authors for a professional masters course in data analytics. The enrolled students are often from all academic backgrounds. MDs, PharmDs, MBAs, etc. The goal of that program is to teach to data story telling in context.

---
## Objectives
 - Learn more forms of data wrangling
 - Learn how to aggregate and group data
 - Learn how to use Groupby with keys 

 ---

Recall that pandas ``Series``s and ``DataFrame``s have four main methods when combining different datasets:

*  `concat()` gives the flexibility to join based on the axis (all rows or all columns)

*  `append()` is the specific case of `concat(axis=0, join='outer')` and creates a new pandas object. Note: this doubles the resources utilized.

*  `join()` is based on the indexes (specified by `set_index`) on the variables `left`,`right`,`inner`, and `outer`

*  `merge()` is based on the user-defined column from each of the two dataframes based on variables like `left_on`, `right_on`, `on`



---

Once we have all our datasets consolidated, how can we work within a dataset?


In [ ]:
# Again, let's import the packages we need
import pandas as pd
import numpy as np

In [ ]:
# And create a toy DataFrame to get us started:
def make_df(cols, ind):
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

make_df('ABCDE', range(5))

,A,B,C,D,E
0,A0,B0,C0,D0,E0
1,A1,B1,C1,D1,E1
2,A2,B2,C2,D2,E2
3,A3,B3,C3,D3,E3
4,A4,B4,C4,D4,E4


In [ ]:
# Here's our class to help display data within this notebook.
class display(object):
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

# Part 3. Aggregation and Grouping

An essential analysis for large data is efficient summarization:  aggregations like ``sum()``, ``mean()``, ``median()``, ``min()``, and ``max()``. Aggregation yields a single number providing insights into a large dataset. Pandas's aggregations include NumPy like ones, and more sophisticated ones based on ``groupby``.

## Simple Aggregation in pandas

The most common built-in pandas aggregation functions include:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |



In [ ]:
# Let's generate a toy Series
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [ ]:
# Aggregation functions like sum() work just as expected with a Series
# As with the NumPy functions, don't worry about memorizing the pandas functions.
# Just know that they exist and where you can find them.
ser.sum()

2.811925491708157

In [ ]:
# DataFrames work a little differently.
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})

df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [ ]:
# By default, aggregation returns results by column
df.mean()

A    0.477888
B    0.443420
dtype: float64

In [ ]:
# Using the `axis` keyword, we can force aggregate by row:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

The pandas method ``describe()`` computes several aggregate values for each column and returns the result. We'll use the sample dataset Planets from the seaborn package:

In [ ]:
import seaborn as sns

planets = sns.load_dataset('planets')

# Before we can call the describe() function, we need to remove the NaN values.
# Note the use of function chaining here:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


## GroupBy: Split, Apply, Combine

Pandas `groupby()` function enables us to perform the “Split-Apply-Combine” data analysis paradigm easily. Basically, we can split a pandas DataFrame into smaller groups using one or more variables and aggregate conditionally on some label or index. The functionality of ``groupby()`` is derived from SQL, for those familiar with that database language.

### Split, apply, combine

A canonical example of split-apply-combine operation, where the "apply" is a summation aggregation:

![](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/03.08-split-apply-combine.png?raw=1)


 ``groupby()`` accomplishes:

- *split* - involves breaking up and grouping ``DataFrame``  on  value of specified key
- *apply*  - computes an aggregate, transformation, or filtering, within individual groups
- *combine* - merges results of these operations into an output array

``groupby()`` can  do all of these steps in a single pass, updating the sum, mean, count, min, or other aggregate for each group, making it more efficient when compared to the earlier operations. 

In [ ]:
# Let's start with a toy DataFrame:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [ ]:
# groupby() takes desired key column as argument:
# Note that there's no actual "output".
# We need to apply an aggregation to this object!
df.groupby('key')

A ``DataFrameGroupBy`` object is returned which is used over and over. Grouping only happens when aggregation is applied. It is called "lazy evaluation". Appropriate apply/combine steps are applied to produce the desired result:

In [ ]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


### Aggregate, filter, transform, apply
 ``GroupBy`` objects have ``aggregate()``, ``filter()``, ``transform()``, and ``apply()`` methods that efficiently implement useful operations before combining the grouped data. To illustrate, we'll use this toy ``DataFrame``:

In [ ]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### Aggregation
``aggregate()`` allows for even more flexibility. It can take a string, a function, or a list thereof, and compute all the aggregates at once:

In [ ]:
df.groupby('key').aggregate(['min', np.median, max,sum])

data1                data2               
      min median max sum   min median max sum
key                                          
A       0    1.5   3   3     3    4.0   5   8
B       1    2.5   4   5     0    3.5   7   7
C       2    3.5   5   7     3    6.0   9  12

Another useful pattern is to pass a dictionary mapping column names to operations to be applied on that column:

In [ ]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


#### Filtering
Filtering allows dropping of data based on group properties.
For example, let's keep groups in which the standard deviation is larger than a threshold value:

In [ ]:
# First, we need to define a function that filters our dataset
def filter_func(x):
    return x['data2'].std() > 4

# Then we call the pandas filter() function using our custom function
display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

#### Transformation

Transformation returns a transformed version of the full data; the output is the same shape as the input. A common example is when we need to normalize the data before performing our analysis. The first step is to center the data by subtracting the group-wise mean using a [lambda function](https://www.w3schools.com/python/python_lambda.asp):

In [ ]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


#### The apply() method

 The pandas ``apply()`` method lets you apply an arbitrary function to the group results.
 
The function should take a ``DataFrame``, and return either a pandas object (e.g., ``DataFrame``, ``Series``) or a scalar; the combine operation will be tailored to the type of output returned. Below is an ``apply()`` that normalizes the first column by the sum of the second:

In [ ]:
def norm_by_data2(x):
    # x is our DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")
df
df.groupby('key').apply(norm_by_data2)

,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3
3,A,0.375000,3
4,B,0.571429,7
5,C,0.416667,9


### Specifying the split key

Till now, we have only split the ``DataFrame`` on a single column name. There are more complex methods where we can arbitrarily define a set of keys:

#### A list, array, series, or index providing the grouping keys

Key can be any series or list with a length matching that of the ``DataFrame``:

In [ ]:
# What happens to the grouping if you change the values in L?
L = [0,1,2,0,1,2]
display('df', 'df.groupby(L).sum()')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(L).sum()
   data1  data2
0      3      8
1      5      7
2      7     12

Thus, there is another and more verbose way of accomplishing the ``df.groupby('key')``:

In [ ]:
display('df', "df.groupby(df['key']).sum()")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(df['key']).sum()
     data1  data2
key              
A        3      8
B        5      7
C        7     12

#### A dictionary or series mapping index to group

In [ ]:
# Another method is to provide a dictionary that maps index values to group keys:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}

display('df2', 'df2.groupby(mapping).sum()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
consonant,12,19
vowel,3,8


#### Any Python function

In [ ]:
# We can also pass any Python function that will input index values and output the group:
display('df2', 'df2.groupby(str.lower).mean()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
a,1.5,4.0
b,2.5,3.5


#### A list of valid keys

Any of the preceding key choices can be combined to group on a multi-index:

In [ ]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0
